# 영업 성공 여부 분류 경진대회

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    make_scorer,
)
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import KFold
from collections import Counter
from catboost import Pool,CatBoostClassifier

## 1. Data Load

In [5]:
df_train = pd.read_csv("/Users/juhyeon/python-workspace/LG-Aimers-4/train.csv")
df_test = pd.read_csv("//Users/juhyeon/python-workspace/LG-Aimers-4/submission.csv")

In [6]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         58317 non-null  object 
 2   business_unit            59299 non-null  object 
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            15338 non-null  object 
 6   enterprise               59299 non-null  object 
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             40566 non-null  object 
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             58358 non-null  object 
 14  product_category      

In [8]:
Asia = ["Australia", "Bangladesh", "Brunei", "Cambodia", "China", "Fiji", "Hong Kong", "India", "Indonesia", "Japan", "Laos",
        "Malaysia", "Maldives", "Mongolia", "Myanmar", "Nepal", "New Zealand", "Papua New Guinea", "Philippines", "Singapore",
        "South Korea", "Sri Lanka", "Taiwan", "Thailand", "Vietnam"]

CIS = ['Belarus', 'Kazakhstan', 'Russia', 'Turkmenistan', 'Ukraine', 'Uzbekistan']

Europe = ["Albania", "Austria", "Belgium", "Bosnia and Herzegovina", 'Bosnia And Herzegovina', "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark",
          "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Isle of Man", "Italy", "Kosovo",
          "Latvia", "Lithuania", "Luxembourg", "Macedonia", "Malta", "Montenegro", "Netherlands", "Norway", "Poland", "Portugal",
          "Romania", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "United Kingdom"]

Latin = ["Anguilla", "Antigua", "Antigua and Barbuda", "Argentina", "Aruba", "Bahamas", "Barbados", "Belize", "British Virgin Islands",
         "Cayman Islands", "Chile", "Colombia", "Costa Rica", "Cuba", "Curacao", "Dominican Republic", "Ecuador", "El Salvador", "Grenada",
         "Guatemala", "Guyana", "Haiti", "Honduras", "Jamaica", "Mexico", "Netherlands Antilles", "Nicaragua", "Panama", "Paraguay", "Peru",
         "Puerto Rico", "Saint Kitts and Nevis", "Saint Lucia", "St Kitts", "St Maarten", "St Vincent", "Suriname", "Trinidad and Tobago",
         "Turks and Caicos Islands", "Uruguay", "US Virgin Islands", "Venezuela", 'Brazil', 'Bolivia']

Middle = ["Afghanistan", "Algeria", "Angola", "Armenia", "Azerbaijan", "Bahrain", "Benin", "Botswana", "Burkina Faso", "Cameroon",
          "Central African Republic", "Congo", "Cote d’lvoire", "Democratic Republic of the Congo", "Djibouti", "Egypt", "Equatorial Guinea",
          "Eritrea", "Ethiopia", "Gabon", "Gambia", "Georgia", "Ghana", "Guinea", "Iran", "Iraq", "Israel", "Ivory Coast", "Jordan", "Kenya",
          "Kuwait", "Lebanon", "Liberia", "Libya", "Malawi", "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Nigeria",
          "Oman", "Pakistan", "Palestine", "Qatar", "Rwanda", "Sao Tome and Principe", "Saudi Arabia", "Senegal", "Seychelles", "Sierra Leone",
          "Somalia", "South Africa", "Sudan", "Swaziland", "Syria", "Togo", "Tunisia", "Türkiye", 'Turkey', "U.A.E", "Uganda", "United Republic of Tanzania",
          "Yemen", "Zambia", "Zimbabwe"]

North = ["Canada", "United States"]

In [9]:
# customer_country 컬럼만 빼내서 처리하고 다시 붙이는 방식!
train_customer_country = df_train['customer_country'].str.rsplit('/', expand=True)
test_customer_country = df_test['customer_country'].str.rsplit('/', expand=True)

# 소문자 & 대문자 & 약어 수정해주기
def check_country_name(s):
    """
    문장의 맨앞은 대문자로, 나머지는 소문자로 변경
    단, and 이나 of 의 경우는 변경하지 않는다.
    """
    if isinstance(s, str):
        words = s.split()
        updated_words = []
        for word in words:
          if word.upper() == 'U.A.E':    # 예외) U.A.E 경우는 그냥 내버려둠
            updated_words.append(word)
          else:
            updated_words.append(word.capitalize() if word.lower() not in ['and', 'of'] else word.lower())
        return ' '.join(updated_words)
    else:
        return s

In [10]:
# train_data
### 국가만 남기기!!
col_train = len(train_customer_country.columns)    # train_customer_country의 컬럼개수

for i in range(col_train):
  train_customer_country[i] = train_customer_country[i].apply(check_country_name)
  train_customer_country[i] = train_customer_country[i].apply(lambda x: x if x in Asia + CIS + Europe + Latin + Middle + North else None)
  
  # new_customer_country[0]의 값이 NaN이 아닌 경우에는 기존 값을 유지
  train_customer_country[0] = train_customer_country[0].combine_first(train_customer_country[i])

# original data 에 적용!
df_train['customer_country'] = train_customer_country[0]

In [11]:
# test_data
### 국가만 남기기!!
col_test = len(test_customer_country.columns)    # test_customer_country의 컬럼개수

for i in range(col_test):
  test_customer_country[i] = test_customer_country[i].apply(check_country_name)
  test_customer_country[i] = test_customer_country[i].apply(lambda x: x if x in Asia + CIS + Europe + Latin + Middle + North else None)
  # new_customer_country[0]의 값이 NaN이 아닌 경우에는 기존 값을 유지
  test_customer_country[0] = test_customer_country[0].combine_first(test_customer_country[i])

# original data 에 적용!
df_test['customer_country'] = test_customer_country[0]

In [12]:
# null 값 존재함!!
df_train['customer_country'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 59299 entries, 0 to 59298
Series name: customer_country
Non-Null Count  Dtype 
--------------  ----- 
55817 non-null  object
dtypes: object(1)
memory usage: 463.4+ KB


## 2. 데이터 전처리

In [13]:
# 바꿔줄 단어들을 리스트로 나열
End = ['End-user', 'End-Customer', 'Commercial end-user']
Specifier = ['Specifier / Influencer']
Solution  = ['Meeting Solution', 'Software / Solution Provider', 'Software/Solution Provider']
Service = ['Authorized Service Center', 'Authorized Service Dealer']
Other = ['Other', 'Others', 'Etc.']

In [14]:
# df_test
df_train.loc[df_train['customer_type'].isin(End), 'customer_type'] = 'End Customer'
df_train.loc[df_train['customer_type'].isin(Specifier), 'customer_type'] = 'Specifier/ Influencer'
df_train.loc[df_train['customer_type'].isin(Solution), 'customer_type'] = 'Solution Eco-Partner'
df_train.loc[df_train['customer_type'].isin(Service), 'customer_type'] = 'Service Partner'
df_train.loc[df_train['customer_type'].isin(Other), 'customer_type'] = 'Other'

# df_test
df_test.loc[df_test['customer_type'].isin(End), 'customer_type'] = 'End Customer'
df_test.loc[df_test['customer_type'].isin(Specifier), 'customer_type'] = 'Specifier/ Influencer'
df_test.loc[df_test['customer_type'].isin(Solution), 'customer_type'] = 'Solution Eco-Partner'
df_test.loc[df_test['customer_type'].isin(Service), 'customer_type'] = 'Service Partner'
df_test.loc[df_test['customer_type'].isin(Other), 'customer_type'] = 'Other'

In [15]:
def assign_id_strategic_ver(row):
    if row['business_unit'] == 'ID' and row['business_area'] in ['corporate / office', 'retail', 'hotel & accommodation', 'education']:
        return 1
    else:
        return 0

df_train['id_strategic_ver'] = df_train.apply(assign_id_strategic_ver, axis=1)
df_test['id_strategic_ver'] = df_test.apply(assign_id_strategic_ver, axis=1)

In [16]:
def assign_it_strategic_ver(row):
    if row['business_unit'] == 'IT' and row['business_area'] in ['corporate / office', 'retail', 'hotel & accommodation', 'education']:
        return 1
    else:
        return 0

df_train['it_strategic_ver'] = df_train.apply(assign_it_strategic_ver, axis=1)
df_test['it_strategic_ver'] = df_test.apply(assign_it_strategic_ver, axis=1)

In [17]:
# 'id_strategic_ver'나 'it_strategic_ver' 중 하나가 1인 경우에는 'idit_strategic_ver'에 1을 할당
df_train['idit_strategic_ver'] = df_train.apply(lambda row: 1 if row['id_strategic_ver'] == 1 or row['it_strategic_ver'] == 1 else 0, axis=1)

# 'id_strategic_ver'나 'it_strategic_ver' 중 하나가 1인 경우에는 'idit_strategic_ver'에 1을 할당
df_test['idit_strategic_ver'] = df_test.apply(lambda row: 1 if row['id_strategic_ver'] == 1 or row['it_strategic_ver'] == 1 else 0, axis=1)


In [18]:
### customer_job과 customer_position 의 값을 배열과 비교하여 변경하기 ###
def search_func_pos(df):
    # 대소문자를 구분하지 않기 위해 소문자로 변환
    df['customer_position'] = df['customer_position'].str.lower()
    df['customer_job'] = df['customer_job'].str.lower()

    # job function과 job seniority 배열 소문자 변환하여 비교
    job_function = ["3d/vfx art", "accounting", "administrative", "arts and design", "business development", "clinical specialist", "community and social services",
                    "consulting", "education", "engineering", "entrepreneurship", "film production", "finance", "graphic/color art", "healthcare services",
                    "human resources", "information technology", "legal", "marketing", "media and communication", "medical imaging specialist", "medical solution",
                    "military and protective services", "operations", "pathologist", "product management", "program and project management",
                    "purchasing", "quality assurance", "radiology professional", "real estate", "research", "sales", "support", "surgery professional", "others", "other" ]

    job_seniority = ["ceo/founder", "partner", "c-level executive", "vice president", "director", "manager",
                     "associate/analyst", "entry level", "trainee", "intern","others"]

    # customer_position과 customer_job을 비교하여 조건에 맞는 행을 선택합니다.
    condition = (df['customer_position'].isin(job_function)) & (df['customer_job'].isin(job_seniority))
    condition_T = df[condition]
    # DataFrame으로 변환
    condition_T = pd.DataFrame(condition_T, columns=['customer_position', 'customer_job'])

    # 조건에 맞는 행의 customer_position과 customer_job 값을 서로 변경합니다.
    df.loc[condition, ['customer_position', 'customer_job']] = df.loc[condition, ['customer_job', 'customer_position']].values

    return df

# 함수를 호출하여 데이터프레임을 변경합니다.
df_train = search_func_pos(df_train)
df_test = search_func_pos(df_test)

In [19]:
### job function에 해당하지 않는 컬럼 확인 ###
# 37개
job_function = ["3d/vfx art", "accounting", "administrative", "arts and design", "business development", "clinical specialist", "community and social services",
                "consulting", "education", "engineering", "entrepreneurship", "film production", "finance", "graphic/color art", "healthcare services",
                "human resources", "information technology", "legal", "marketing", "media and communication", "medical imaging specialist", "medical solution",
                "military and protective services", "operations", "pathologist", "product management", "program and project management",
                "purchasing", "quality assurance", "radiology professional", "real estate", "research", "sales", "support", "surgery professional", "others", "other" ]

# customer_job 컬럼에서 job_function에 해당하는 카테고리를 필터링
function_job = set(job_function)
matching_job = set(df_train['customer_job']).intersection(function_job)

# job_function에 해당하지 않는 나머지 카테고리들을 출력
other_job = set(df_train['customer_job']) - matching_job

In [20]:
# 각 카테고리에 대한 매핑 딕셔너리 생성
three_d_vfx_art = ["3d/vfx art"]

accounting = ["accounting", "accounts payable", "account exec/manager", "account management"]

administrative = ["administrative", "platform administrator", "imaging administrator", "pacs administrator", "systems administrator", "facility administrator", "network administrator", "admin", "admin assistant", "administración", "administration", "administrative assistant", "adminisztráció", "amministrativo"]

arts_and_design = ["arts and design", "support/facilitator, designer", "design and provide equipment", "design/build", "design/decision maker", "design/install/training/support", "kreation und design", "kreation_und_design", "lead designer", "művészet_és_design", "interior designer", "art and design", "art installation", "arte y diseño", "arte_e_design", "artist, lead on equipment selection", "arts_and_design", "design", "designer", "designer, creative technologist", "designer, producer", "designer/installer", "designere / budget", "designers"]

business_development = ["business development", "sourcing & quoting for end user", "distributor quotation", "developer/property", "curation", "quotation curator", "quote gathering/proposer to owner", "quoting project", "business_development", "development coordinator/procurement"]

clinical_specialist = ["clinical specialist", "clinic", "mental health"]

community_and_social_services = ["community and social services", "community_and_social_services"]

consulting = ["consulting", "technical advisor, reseller", "consultent", "consultant", "consultant / purchaser", "consultant,cabinet fabricator", "strategic communications", "strategy & operations specialist", "solution advisor", "solutions architect", "technology consultant"]

education = ["education", "teacher", "teaching", "educator", "k12 school", "higher education (college & university)", "institute & academy", "instructor"]

engineering = ["engineering", "tech", "technical", "electrical contractor", "implement", "senior design engineer", "solution engineer", "system engineer", "systems engineer", "lead engineer", "electronics & telco", "engineer", "engineering & technical", "engineering & technical executive", "engineering director", "engineering, design, and install", "system designer, integrator", "systems design", "systems designer", "principal engineer", "hardware", "hardware design engineer", "hardware selection", "chief eng.", "chief engineer", "chief of engineering", "design engineer", "director of engineering"]

entrepreneurship = ["business owner", "ceo", "director comercial", "head", "engagement executive", "execution", "executive", "owner","owner representation", "owning company", "ownner-marketing director", "ceo/founder", "lead", "organizer", "leader", "vice president", "vp/gm", "underboss", "the big boss"]

film_production = ["film production", "home theater", "community theater"]

finance = ["finance", "finanzas", "finanzen", "pénzügy", "finance executive"]

graphic_color_art = ["graphic/color art", "colorist", "gc", "graphic design"]

healthcare_services = ["healthcare services", "healthcare professionals", "healthcare_services", "healthcare"]

human_resources = ["human resources", "hr posting", "hr", "human_resources"]

information_technology = ["information technology", "software developer", "emerging technology / innovation", "informatics, touch capability", "information technology\u200b", "information_technology", "infrastructure", "it", "it - information technology", "it admin", "it administrator", "it dairector", "it department", "it director", "it hardware technician", "it integrator", "it manager", "it project lead", "it specialist", "it support", "it tech.", "it/software", "application development", "cloud / mobility", "collaboration & web apps", "computing & it"],

legal = ["legal"]

marketing = ["marketing", "technical marketing", "advertising", "product marketing", "advertising and promotions team", "event marketing", "field marketing", "marketing coordinator", "marketing executive", "marketing operations"]

media_and_communication = ["media and communication", "broadcasting & media", "media and communications", "media_and_communication", "media_e_comunicazione", "medien_und_kommunikation", "medios_de_comunicación", "média_és_kommunikáció"]

medical_imaging_specialist = ["medical imaging specialist", "medical imaging  specialist", "spécialiste_en_imagerie_médicale"]

medical_solution = ["medical solution", "doctor", "tierarzt", "medical solution  provider", "medical solution provider", "medical solution provider\u200b"]

military_and_protective_services = ["military and protective services", "military_and_protective_services"]

operations = ["coo", "director of operations","regional director of operations", "operations executive", "operations manager", "operations", "facilities and operations", "üzemeltetés"]

pathologist = ["pathologist"]

product_management = ["product management", "product_management"]

program_and_project_management = ["program and project management", "av project manager", "signage subcontractor p/m", "general manager - project manager", "digital project manager", "program directors", "gestión_de_proyectos", "program-_és_projektmenedzsment", "program_and_project_management", "program_and_project_manager", "programm- und projektmanagement", "programm-_und_projektmanagement", "project administrator", "project coordinator", "project director", "project facilitator", "project head", "project lead", "project manage", "project manager", "project manager / estimator", "project manager / principal", "project manager/designer", "project researcher", "project sales/manage", "projection manager", "projectr mgmt", "projektmenedzsment\tprogram and project management", "planner", "planner/purchaser", "planning and installation", "pm", "a/v project manager", "project manager", "owner / project manager", "producer/project manager"]

purchasing = ["purchasing", "buyer", "buyer, coordinating", "obtain quotes, process purchase", "requirements and buyer", "ordering manager", "requisition", "purchase", "purchase and install", "purchase dept", "purchaser", "purchaser, it and installer", "purchasers", "purchasing agent", "purchasing authority", "purchasing coordinator", "purchasing director", "purchasing manager", "purchasing supervisor", "purchsing", "director purchaser", "drop, purchase maxhub", "public bidder", "bidder"]

quality_assurance = ["quality assurance", "quality_assurance"]

radiology_professional = ["radiology professional", "profesional de radiología", "radiology  professional", "radiology_professional"]

real_estate = ["real estate", "building owner", "property owner"]

research = ["research", "associate/analyst", "r&d project manager", "research & development", "research and developement", "research products and prices", "research/install", "product research", "product researcher"]

sales = ["sales", "asking for quote for client", "field / outside sales", "sourcing / procurement", "sourcing/procurement", "reseller/integrator", "procurement", "procurement specialist", "procurment", "revendedor", "car dealership", "vendor / reseller", "vendite", "értékesítés", "technical sales", "reseller", "sale", "sales engineering", "sales executive", "sales manager", "sales operations", "sales rep", "salesman"]

support = ["support", "help desk / desktop services", "helpdesk specialist", "post install support and service", "supplier and installation"]

surgery_professional = ["surgery professional", "profesional de cirugía", "surgery professional\u200b"]


In [21]:
df_train.loc[df_train['customer_job'].isin(three_d_vfx_art), 'customer_job'] = 'three_d_vfx_art'
df_train.loc[df_train['customer_job'].isin(accounting), 'customer_job'] = 'accounting'
df_train.loc[df_train['customer_job'].isin(administrative), 'customer_job'] = 'administrative'
df_train.loc[df_train['customer_job'].isin(arts_and_design), 'customer_job'] = 'arts_and_design'
df_train.loc[df_train['customer_job'].isin(business_development), 'customer_job'] = 'business_development'
df_train.loc[df_train['customer_job'].isin(clinical_specialist), 'customer_job'] = 'clinical_specialist'
df_train.loc[df_train['customer_job'].isin(community_and_social_services), 'customer_job'] = 'community_and_social_services'
df_train.loc[df_train['customer_job'].isin(consulting), 'customer_job'] = 'consulting'
df_train.loc[df_train['customer_job'].isin(education), 'customer_job'] = 'education'
df_train.loc[df_train['customer_job'].isin(engineering), 'customer_job'] = 'engineering'
df_train.loc[df_train['customer_job'].isin(entrepreneurship), 'customer_job'] = 'entrepreneurship'
df_train.loc[df_train['customer_job'].isin(film_production), 'customer_job'] = 'film_production'
df_train.loc[df_train['customer_job'].isin(finance), 'customer_job'] = 'finance'
df_train.loc[df_train['customer_job'].isin(film_production), 'customer_job'] = 'film_production'
df_train.loc[df_train['customer_job'].isin(graphic_color_art), 'customer_job'] = 'graphic_color_art'
df_train.loc[df_train['customer_job'].isin(healthcare_services), 'customer_job'] = 'healthcare_services'
df_train.loc[df_train['customer_job'].isin(human_resources), 'customer_job'] = 'human_resources'
df_train.loc[df_train['customer_job'].isin(information_technology), 'customer_job'] = 'information_technology'
df_train.loc[df_train['customer_job'].isin(legal), 'customer_job'] = 'legal'
df_train.loc[df_train['customer_job'].isin(marketing), 'customer_job'] = 'marketing'
df_train.loc[df_train['customer_job'].isin(media_and_communication), 'customer_job'] = 'media_and_communication'
df_train.loc[df_train['customer_job'].isin(medical_imaging_specialist), 'customer_job'] = 'medical_imaging_specialist'
df_train.loc[df_train['customer_job'].isin(medical_solution), 'customer_job'] = 'medical_solution'
df_train.loc[df_train['customer_job'].isin(military_and_protective_services), 'customer_job'] = 'military_and_protective_services'
df_train.loc[df_train['customer_job'].isin(operations), 'customer_job'] = 'operations'
df_train.loc[df_train['customer_job'].isin(pathologist), 'customer_job'] = 'pathologist'
df_train.loc[df_train['customer_job'].isin(product_management), 'customer_job'] = 'product_management'
df_train.loc[df_train['customer_job'].isin(program_and_project_management), 'customer_job'] = 'program_and_project_management'
df_train.loc[df_train['customer_job'].isin(purchasing), 'customer_job'] = 'purchasing'
df_train.loc[df_train['customer_job'].isin(quality_assurance), 'customer_job'] = 'quality_assurance'
df_train.loc[df_train['customer_job'].isin(radiology_professional), 'customer_job'] = 'radiology_professional'
df_train.loc[df_train['customer_job'].isin(real_estate), 'customer_job'] = 'real_estate'
df_train.loc[df_train['customer_job'].isin(research), 'customer_job'] = 'research'
df_train.loc[df_train['customer_job'].isin(sales), 'customer_job'] = 'sales'
df_train.loc[df_train['customer_job'].isin(support), 'customer_job'] = 'support'
df_train.loc[df_train['customer_job'].isin(surgery_professional), 'customer_job'] = 'surgery_professional'

In [22]:
df_test.loc[df_test['customer_job'].isin(three_d_vfx_art), 'customer_job'] = 'three_d_vfx_art'
df_test.loc[df_test['customer_job'].isin(accounting), 'customer_job'] = 'accounting'
df_test.loc[df_test['customer_job'].isin(administrative), 'customer_job'] = 'administrative'
df_test.loc[df_test['customer_job'].isin(arts_and_design), 'customer_job'] = 'arts_and_design'
df_test.loc[df_test['customer_job'].isin(business_development), 'customer_job'] = 'business_development'
df_test.loc[df_test['customer_job'].isin(clinical_specialist), 'customer_job'] = 'clinical_specialist'
df_test.loc[df_test['customer_job'].isin(community_and_social_services), 'customer_job'] = 'community_and_social_services'
df_test.loc[df_test['customer_job'].isin(consulting), 'customer_job'] = 'consulting'
df_test.loc[df_test['customer_job'].isin(education), 'customer_job'] = 'education'
df_test.loc[df_test['customer_job'].isin(engineering), 'customer_job'] = 'engineering'
df_test.loc[df_test['customer_job'].isin(entrepreneurship), 'customer_job'] = 'entrepreneurship'
df_test.loc[df_test['customer_job'].isin(film_production), 'customer_job'] = 'film_production'
df_test.loc[df_test['customer_job'].isin(finance), 'customer_job'] = 'finance'
df_test.loc[df_test['customer_job'].isin(film_production), 'customer_job'] = 'film_production'
df_test.loc[df_test['customer_job'].isin(graphic_color_art), 'customer_job'] = 'graphic_color_art'
df_test.loc[df_test['customer_job'].isin(healthcare_services), 'customer_job'] = 'healthcare_services'
df_test.loc[df_test['customer_job'].isin(human_resources), 'customer_job'] = 'human_resources'
df_test.loc[df_test['customer_job'].isin(information_technology), 'customer_job'] = 'information_technology'
df_test.loc[df_test['customer_job'].isin(legal), 'customer_job'] = 'legal'
df_test.loc[df_test['customer_job'].isin(marketing), 'customer_job'] = 'marketing'
df_test.loc[df_test['customer_job'].isin(media_and_communication), 'customer_job'] = 'media_and_communication'
df_test.loc[df_test['customer_job'].isin(medical_imaging_specialist), 'customer_job'] = 'medical_imaging_specialist'
df_test.loc[df_test['customer_job'].isin(medical_solution), 'customer_job'] = 'medical_solution'
df_test.loc[df_test['customer_job'].isin(military_and_protective_services), 'customer_job'] = 'military_and_protective_services'
df_test.loc[df_test['customer_job'].isin(operations), 'customer_job'] = 'operations'
df_test.loc[df_test['customer_job'].isin(pathologist), 'customer_job'] = 'pathologist'
df_test.loc[df_test['customer_job'].isin(product_management), 'customer_job'] = 'product_management'
df_test.loc[df_test['customer_job'].isin(program_and_project_management), 'customer_job'] = 'program_and_project_management'
df_test.loc[df_test['customer_job'].isin(purchasing), 'customer_job'] = 'purchasing'
df_test.loc[df_test['customer_job'].isin(quality_assurance), 'customer_job'] = 'quality_assurance'
df_test.loc[df_test['customer_job'].isin(radiology_professional), 'customer_job'] = 'radiology_professional'
df_test.loc[df_test['customer_job'].isin(real_estate), 'customer_job'] = 'real_estate'
df_test.loc[df_test['customer_job'].isin(research), 'customer_job'] = 'research'
df_test.loc[df_test['customer_job'].isin(sales), 'customer_job'] = 'sales'
df_test.loc[df_test['customer_job'].isin(support), 'customer_job'] = 'support'
df_test.loc[df_test['customer_job'].isin(surgery_professional), 'customer_job'] = 'surgery_professional'

In [23]:
# 제품군 간단한 전처리 하기

# 각 카테고리에 대한 매핑 딕셔너리 생성
other = ["other", "others", "etc.", "khác", "outros", "lainnya", "אחר", "otros"]
commercial_tv = ["commercial tv", "commercial tv,tv", "commercial tv,audio/video", "commercial_tv", "tv,commercial tv", "comercial tv"]
heating = ["heating", "חימום" ,"حلول التدفئة", "isıtma", "ogrzewanie (pompy ciepła)", "calefacción"]
multi_split  = ["multi-split", "פיצול מרובה", "multi split"]
single_split = ["single-split", "split tunggal", "single split"]
chiller = ["chiller", "مبرد (تشيلر)", "soğutucu", "pendingin"]
video_wall_signage  = ["video wall signage", "videwall", "video wall"]
hotel_tv = ["hotel tv", "酒店電視"]
hospital_tv = ["hospital tv", "醫院電視"]


df_train.loc[df_train['product_category'].isin(other), 'product_category'] = 'other'
df_train.loc[df_train['product_category'].isin(commercial_tv), 'product_category'] = 'commercial_tv'
df_train.loc[df_train['product_category'].isin(heating), 'product_category'] = 'heating'
df_train.loc[df_train['product_category'].isin(multi_split), 'product_category'] = 'multi_split'
df_train.loc[df_train['product_category'].isin(single_split), 'product_category'] = 'single_split'
df_train.loc[df_train['product_category'].isin(chiller), 'product_category'] = 'chiller'
df_train.loc[df_train['product_category'].isin(video_wall_signage), 'product_category'] = 'video_wall_signage'
df_train.loc[df_train['product_category'].isin(hotel_tv), 'product_category'] = 'hotel_tv'

df_test.loc[df_test['product_category'].isin(other), 'product_category'] = 'other'
df_test.loc[df_test['product_category'].isin(commercial_tv), 'product_category'] = 'commercial_tv'
df_test.loc[df_test['product_category'].isin(heating), 'product_category'] = 'heating'
df_test.loc[df_test['product_category'].isin(multi_split), 'product_category'] = 'multi-split'
df_test.loc[df_test['product_category'].isin(single_split), 'product_category'] = 'single_split'
df_test.loc[df_test['product_category'].isin(chiller), 'product_category'] = 'chiller'
df_test.loc[df_test['product_category'].isin(video_wall_signage), 'product_category'] = 'video_wall_signage'
df_test.loc[df_test['product_category'].isin(hotel_tv), 'product_category'] = 'hotel_tv'

In [24]:
import re

def custom_replace(value):
    value_str = str(value)  # 정수형 데이터를 문자열로 변환
    if "less" in value_str.lower():
        return 'Less than 3 months'
    elif "3" in value_str and "6" in value_str:
        return '3 Months ~ 6 Months'
    elif "6" in value_str and "9" in value_str:
        return '6 Months ~ 9 Months'
    elif "9" in value_str and "1" in value_str:
        return '9 Months ~ 1 year'
    elif "more" in value_str.lower():
        return 'More than a year'
    else: # 941개 NaN값 처리
#         return -1 # 전부 -1
        return 'Not specified' #전부 Not specified
#         return value  #원래 결측치 값 -> 이후 결측치 0으로 채우기


# df_train/df_test에 함수 적용
df_train['expected_timeline'] = df_train['expected_timeline'].apply(custom_replace)
df_test['expected_timeline'] = df_test['expected_timeline'].apply(custom_replace)

In [25]:
### 결측치 처리 함수 작성 ###
def ver_cus_0to1(df):
  df.loc[(df['business_area'].isin(['corporate / office', 'retail', 'education', 'hotel & accommodation'])) &
         (df['customer_type'].isin(['End-Customer', 'End Customer', 'End-user'])),
         'ver_cus'] = 1
  return df

### df_train/df_test에 적용
df_train = ver_cus_0to1(df_train)
df_test = ver_cus_0to1(df_test)

In [26]:
### 결측치 처리 함수 ###
def ver_pro_0to1(df):
    df.loc[(df['business_area'].isin(['corporate / office', 'retail', 'hotel & accommodation','education'])) &
           (df['product_category'].str.contains('signage', case=False) |
            df['product_category'].str.contains('hotel tv', case=False)),
           'ver_pro'] = 1
    return df

### df_train/df_test에 적용
df_train = ver_pro_0to1(df_train)
df_test = ver_pro_0to1(df_test)

In [27]:
# 확인해보기
df_train.iloc[1:6, df_train.columns.isin(['business_unit', 'com_reg_ver_win_rate', 'business_area', 'product_category', 'customer_type', 'ver_pro', 'ver_cus'])]

,business_unit,com_reg_ver_win_rate,customer_type,product_category,ver_cus,ver_pro,business_area
1,AS,0.066667,End Customer,multi_split,1,0,corporate / office
2,AS,0.088889,End Customer,single_split,1,0,corporate / office
3,AS,0.088889,End Customer,vrf,1,0,corporate / office
4,AS,0.088889,Specifier/ Influencer,multi_split,0,0,corporate / office
5,AS,0.040816,End Customer,chiller,1,0,corporate / office


In [28]:
# 불필요해 보이는 열 제거
#columns_to_drop = ["customer_country", "customer_country.1", "response_corporate","product_subcategory", "product_modelname", "business_subarea"]
columns_to_drop = ["customer_country.1", "response_corporate","product_subcategory", "product_modelname", "business_subarea"]

df_train = df_train.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)

In [29]:
is_holdout = False
iterations = 3000   # 최대학습수
patience = 100      # 100번동안 성능개선 없으면 조기종료! (숫자 낮춰볼 수도 있을듯!)

# Train/Val 데이터를 5-fold로 나누기
kf = KFold(n_splits=5, shuffle=True, random_state=1117)

# 각 fold에 대한 예측값을 저장할 리스트
ensemble_preds = []

## 3. 모델 학습

In [30]:
# 클래스 0과 클래스 1의 비율에 따라 scale_pos_weight 설정
df_train['is_converted'].value_counts()

scale_pos_weight = 54449/4850
scale_pos_weight    # 11.22

11.22659793814433

In [31]:
# 범주형 데이터 정의
# target encoding & one-hot encoding 해줌

cat_features = ['bant_submit','customer_country','business_unit', 'customer_idx', 'customer_type', 'enterprise', 'customer_job', 'inquiry_type', 'product_category', 'customer_position', 'business_area', 'lead_owner', 'expected_timeline']  # 실제 범주형 특성 이름으로 변경

In [32]:
# CatBoost 학습
best_models = []

models = []
for train_index, val_index in kf.split(df_train):
    print("="*50)
    
    # Train/Val 데이터 분할
    x_train_fold, x_val_fold = df_train.drop("is_converted", axis=1).iloc[train_index], df_train.drop("is_converted", axis=1).iloc[val_index]
    y_train_fold, y_val_fold = df_train["is_converted"].iloc[train_index], df_train["is_converted"].iloc[val_index]

    x_train_fold[cat_features] = x_train_fold[cat_features].astype('str')
    x_val_fold[cat_features] = x_val_fold[cat_features].astype('str')

    
    # Train 데이터 imputation
    imputer = SimpleImputer(strategy='most_frequent')
    x_train_fold_imputed = pd.DataFrame(imputer.fit_transform(x_train_fold), columns=x_train_fold.columns)

    # Validation 데이터 imputation 및 예측
    x_val_fold_imputed = pd.DataFrame(imputer.transform(x_val_fold), columns=x_val_fold.columns)
    
    x_train_fold_imputed[cat_features] = x_train_fold_imputed[cat_features].astype('str')
    x_val_fold_imputed[cat_features] = x_val_fold_imputed[cat_features].astype('str')
    
    # 모델학습
    model = CatBoostClassifier(iterations=iterations,
                               random_state=1117,
                               scale_pos_weight = 11, 
                               eval_metric="F1",
                               cat_features=cat_features, one_hot_max_size=4)
    
    model.fit(x_train_fold_imputed, y_train_fold, eval_set=[(x_val_fold_imputed, y_val_fold)], 
              early_stopping_rounds=patience, verbose = 100)
     
    fold_pred = model.predict(x_val_fold_imputed)
    
    # fold별 예측값 저장
    ensemble_preds.append(fold_pred)
    best_models.append(model)

    # fold별 confusion matrix 계산
    fold_pred_binary = (fold_pred == 'True').astype(int)    # fold_pred를 T/F 형식으로 변환
    y_val_fold_binary = y_val_fold.astype(int)              # T/F 형식의 레이블을 0 / 1로 변환

    # Confusion matrix & F1 score 계산
    cm = confusion_matrix(y_val_fold_binary, fold_pred_binary)
    print("Confusion Matrix:")
    print(cm)

    f1 = f1_score(y_val_fold_binary, fold_pred_binary)
    print(f"F1 Score: {f1:.4f}")

    if is_holdout:
        break    

Learning rate set to 0.050976
0:	learn: 0.8445550	test: 0.8609147	best: 0.8609147 (0)	total: 101ms	remaining: 5m 2s
100:	learn: 0.9423212	test: 0.9478988	best: 0.9487843 (77)	total: 3.6s	remaining: 1m 43s
200:	learn: 0.9501661	test: 0.9511699	best: 0.9516151 (191)	total: 6.95s	remaining: 1m 36s
300:	learn: 0.9560203	test: 0.9512466	best: 0.9516980 (208)	total: 10.7s	remaining: 1m 35s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9516979735
bestIteration = 208

Shrink model to first 209 iterations.
Confusion Matrix:
[[10371   505]
 [   49   935]]
F1 Score: 0.7715
Learning rate set to 0.050976
0:	learn: 0.8533483	test: 0.8500073	best: 0.8500073 (0)	total: 36.7ms	remaining: 1m 49s
100:	learn: 0.9431015	test: 0.9500166	best: 0.9500689 (68)	total: 3.77s	remaining: 1m 48s
200:	learn: 0.9503344	test: 0.9533856	best: 0.9539805 (176)	total: 7.35s	remaining: 1m 42s
300:	learn: 0.9549937	test: 0.9531027	best: 0.9542496 (237)	total: 11s	remaining: 1m 39s
Stopped by overfitti

In [34]:
# 여기에 fold별 예측값 저장되어 있음
ensemble_preds

[array(['False', 'True', 'False', ..., 'False', 'False', 'False'],
       dtype=object),
 array(['True', 'True', 'True', ..., 'False', 'False', 'False'],
       dtype=object),
 array(['True', 'False', 'False', ..., 'False', 'False', 'False'],
       dtype=object),
 array(['True', 'True', 'True', ..., 'False', 'False', 'False'],
       dtype=object),
 array(['True', 'True', 'True', ..., 'False', 'False', 'False'],
       dtype=object)]

In [35]:
for i, preds in enumerate(ensemble_preds, 1):
    num_true = np.sum(preds == 'True')
    total_samples = len(preds)
    true_ratio = num_true / total_samples

    print(f"Array {i}:")
    print(f"Number of True: {num_true}")
    print(f"True Ratio: {true_ratio:.2%}\n")


Array 1:
Number of True: 1440
True Ratio: 12.14%

Array 2:
Number of True: 1414
True Ratio: 11.92%

Array 3:
Number of True: 1342
True Ratio: 11.32%

Array 4:
Number of True: 1374
True Ratio: 11.59%

Array 5:
Number of True: 1411
True Ratio: 11.90%



In [36]:
for idx, preds in enumerate(ensemble_preds):
    print(f"Fold {idx+1} 예측값 개수:", len(preds))

Fold 1 예측값 개수: 11860
Fold 2 예측값 개수: 11860
Fold 3 예측값 개수: 11860
Fold 4 예측값 개수: 11860
Fold 5 예측값 개수: 11859


In [37]:
# 앙상블을 통한 최종 예측
# 5개 모델 중 2개 이상 모델이 1로 예측할 경우

final_preds = []

for i in range(min(map(len, ensemble_preds))):
    # 각 행별로, 모든 fold의 예측값을 가져와서, 해당 위치에서 1(True)로 예측한 모델의 개수를 세서 놓는 list
    combined_preds = [ensemble_preds[j][i] == 'True' for j in range(len(ensemble_preds)) if len(ensemble_preds[j]) > i]

    # 모든 fold에서 예측값이 있는 경우에만 다수결 적용
    if combined_preds:
        num_ones = sum(combined_preds)  # 1로 분류된 모델의 개수를 세기 위해 1의 개수를 계산
        if num_ones >= 2:               # 2개 이상의 모델이 1로 분류했을 때
          final_preds.append(True)
        else:
          final_preds.append(False)     # 그 외의 경우에는 0으로 예측

In [38]:
# 최종 앙상블 모델
num_true_final = np.sum(final_preds)
total_samples_final = len(final_preds)
true_ratio_final = num_true_final / total_samples_final

print(f"Number of True: {num_true_final}")
print(f"True Ratio: {true_ratio_final:.2%}")


Number of True: 1549
True Ratio: 13.06%


### 모델 성능 보기

In [39]:
# 맞는 공식인지 모르겠음
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [40]:
# Validation 데이터에 대한 평가
get_clf_eval(y_val_fold, final_preds)

오차행렬:
 [[  811   183]
 [  738 10127]]

정확도: 0.9223
정밀도: 0.5236
재현율: 0.8159
F1: 0.6378


## 4. Submission Data

In [41]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)
x_test[cat_features] = x_test[cat_features].astype('str')


# 결측치를 최빈값으로 대체
imputer = SimpleImputer(strategy='most_frequent')
x_test_imputed = pd.DataFrame(imputer.fit_transform(x_test), columns=x_test.columns)

x_test_imputed[cat_features] = x_test_imputed[cat_features].astype('str')

In [42]:
# Test 데이터 예측
test_preds = []
for model in best_models:
    test_pred = model.predict(x_test_imputed)
    test_preds.append(test_pred)

In [43]:
# 앙상블을 통한 최종 예측/5개 모델 중 2개 이상 모델이 1로 예측할 경우        
final_test_preds = []

for i in range(len(test_preds[0])):
    # combined_test_preds = [test_preds[j][i] for j in range(len(test_preds))]
    combined_test_preds = [test_preds[j][i] == 'True' for j in range(len(test_preds))]
    num_ones = sum(combined_test_preds)  # 1로 분류된 모델의 개수를 세기 위해 True의 개수를 계산
    if num_ones >= 2:  # 2개 이상의 모델이 True로 분류했을 때
        final_test_preds.append(True)
    else:
        final_test_preds.append(False)  # 그 외의 경우에는 False로 예측


In [44]:
sum(final_test_preds) # True로 예측된 개수

2302

In [45]:
# Submission 파일에 모델 예측값 추가
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = final_test_preds
df_sub.to_csv("submission_1.csv", index=False)